The following code is commented due to that it cannot be run from a cloned version of the repository directly as the IMDb datasets are past the GitHub 100 MB limit. To be able to reproduce this section of code refer to the README, to obtain the data and folder structure needed to do so.

``` python
# Set movie name as index
movie_data = movie_data.set_index('movie_name')

# Load IMDb datasets
akas = pd.read_csv('./data/imdb/title_akas.tsv', sep = "\t").dropna()
ratings = pd.read_csv('./data/imdb/title_ratings.tsv', sep = "\t")

# Set akas titles to lowercase and strip any stray whitespace
akas.title = akas.title.apply(lambda x: x.lower().strip())

akas.drop_duplicates(subset = 'title', inplace = True)

# Take only features used for future merge
akas = akas[["titleId", "title"]].set_index('title')

# Merge and drop dup
movie_data = pd.merge(movie_data, akas, left_index= True, right_index = True)

# Merge scores based on merged IMDb indexes
movie_data = pd.merge(movie_data, ratings, left_on = 'titleId', right_on = 'tconst')

# Drop unused indexes
movie_data.drop(['titleId', 'tconst'], axis = 1, inplace = True)
```

The result is saved in the following imported dataset.

In [3]:
import pandas as pd
movie_data = pd.read_csv('./data/movie_data_imdbscores.csv')
movie_data.head()

,movie_id,freebase_movie_id,release_date,box_office_revenue,runtime,languages,countries,genres,english language,german language,...,Romance Film,Romantic drama,Comedy film,Documentary,plot_summary,F_gender_porportion,M_gender_porportion,actor_age_at_movie_release,averageRating,numVotes
0,30332673,/m/0crs0hx,2010-01-01,NaN,90.0,[''],['united states of america'],"['Comedy', 'Comedy film', 'Sex comedy', 'Sport...",False,False,...,False,False,True,False,Two horny college guys get summer jobs at a ch...,0.666667,0.333333,38.333333,3.7,3222
1,4213160,/m/0bq8q8,1971-12-17,NaN,119.0,['english language'],['united states of america'],"['Action', 'Action/Adventure', 'Comedy', 'Crim...",True,False,...,False,False,False,False,"Set in Hamburg, West Germany, several criminal...",0.250000,0.750000,41.625000,6.3,2631
2,20624798,/m/05222ld,2008-01-01,NaN,78.0,['english language'],"['australia', 'israel']","['Animation', 'Drama', 'Indie', 'Stop motion',...",True,False,...,False,False,False,False,The film mainly focuses on 28-year-old Dave Pe...,0.111111,0.888889,41.250000,7.2,22
3,2250713,/m/06z7m4,1988-01-01,NaN,98.0,['english language'],"['hungary', 'united states of america']","['Coming of age', 'Drama', 'Family Drama', 'Pe...",True,False,...,False,False,False,False,The father escaped the Soviet invasion of Buda...,0.000000,1.000000,42.000000,5.9,82
4,25969588,/m/02pmmn1,2007-01-01,NaN,NaN,['english language'],['united states of america'],"['Comedy', 'Drama', 'Indie']",True,False,...,False,False,False,False,NaN,0.200000,0.800000,42.000000,6.4,346


The final dataset features is as follows:

In [4]:
movie_data.columns

Index(['movie_id', 'freebase_movie_id', 'release_date', 'box_office_revenue',
       'runtime', 'languages', 'countries', 'genres', 'english language',
       'german language', 'silent film language', 'spanish language',
       'japanese language', 'italian language', 'tamil language',
       'hindi language', 'malayalam language', 'mandarin language',
       'french language', 'Action', 'Adventure', 'Horror', 'Thriller', 'Drama',
       'Crime Fiction', 'Black-and-white', 'Comedy', 'Indie', 'Short Film',
       'Silent film', 'Family Film', 'World cinema', 'Musical',
       'Action/Adventure', 'Romance Film', 'Romantic drama', 'Comedy film',
       'Documentary', 'plot_summary', 'F_gender_porportion',
       'M_gender_porportion', 'actor_age_at_movie_release', 'averageRating',
       'numVotes'],
      dtype='object')

We proceed to drop unused variables for the further analyzes, such as features already processed for feature extraction, features with high missing values density (such as box office revenue) and repetitive features such as gender proportion (we only keep one).

In [5]:
movie_data = movie_data.drop(["freebase_movie_id", "languages", "genres", "box_office_revenue"], axis = 1)

In [6]:
movie_data.head()

,movie_id,release_date,runtime,countries,english language,german language,silent film language,spanish language,japanese language,italian language,...,Romance Film,Romantic drama,Comedy film,Documentary,plot_summary,F_gender_porportion,M_gender_porportion,actor_age_at_movie_release,averageRating,numVotes
0,30332673,2010-01-01,90.0,['united states of america'],False,False,False,False,False,False,...,False,False,True,False,Two horny college guys get summer jobs at a ch...,0.666667,0.333333,38.333333,3.7,3222
1,4213160,1971-12-17,119.0,['united states of america'],True,False,False,False,False,False,...,False,False,False,False,"Set in Hamburg, West Germany, several criminal...",0.250000,0.750000,41.625000,6.3,2631
2,20624798,2008-01-01,78.0,"['australia', 'israel']",True,False,False,False,False,False,...,False,False,False,False,The film mainly focuses on 28-year-old Dave Pe...,0.111111,0.888889,41.250000,7.2,22
3,2250713,1988-01-01,98.0,"['hungary', 'united states of america']",True,False,False,False,False,False,...,False,False,False,False,The father escaped the Soviet invasion of Buda...,0.000000,1.000000,42.000000,5.9,82
4,25969588,2007-01-01,NaN,['united states of america'],True,False,False,False,False,False,...,False,False,False,False,NaN,0.200000,0.800000,42.000000,6.4,346


In [7]:
movie_data.columns

Index(['movie_id', 'release_date', 'runtime', 'countries', 'english language',
       'german language', 'silent film language', 'spanish language',
       'japanese language', 'italian language', 'tamil language',
       'hindi language', 'malayalam language', 'mandarin language',
       'french language', 'Action', 'Adventure', 'Horror', 'Thriller', 'Drama',
       'Crime Fiction', 'Black-and-white', 'Comedy', 'Indie', 'Short Film',
       'Silent film', 'Family Film', 'World cinema', 'Musical',
       'Action/Adventure', 'Romance Film', 'Romantic drama', 'Comedy film',
       'Documentary', 'plot_summary', 'F_gender_porportion',
       'M_gender_porportion', 'actor_age_at_movie_release', 'averageRating',
       'numVotes'],
      dtype='object')

In [8]:
movie_data.plot_summary[0]

"Two horny college guys get summer jobs at a cheerleader camp for the summer. Friendly and naive Michael is mistaken as a homosexual even though he isn't. World class jerk Andy has an addiction to masturbation and also tries to hook up with a different girl every night. Michael and Andy help out head cheerleader Sophie form her own cheerleading team. A group of strippers end up becoming cheerleaders after Michael and Andy have a run-in with them at a local gentlemen's club. A college scholarship is offered to the group of cheerleaders that wins the climatic team competition."

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import spacy
import string 
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation as LDA

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


import matplotlib.pyplot as plt
%matplotlib inline

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [10]:
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words
stopwords_dict = Counter(spacy_stopwords)

In [11]:

lemmatizer =WordNetLemmatizer()
#cleaning "pipeline"  
def clean_summary(text, stopwords, lemmatization):

      
      #perform casefolding
      text =text.casefold()

      #remove punctuations for each twitter
      text = ' '.join(text_ for text_ in text.split() if text_ not in string.punctuation)

      #remove multiply commas and dots everywhere in tweets      
      text = re.sub('\.|,*','', text)
      
      if stopwords:
          #remove the stopwords
          text = ' '.join([word for word in text.split() if word not in stopwords_dict])         
            
      if lemmatization :
          #perform lemmatization
          text = ' '.join(lemmatizer.lemmatize(text_)  for text_ in text.split() )

      #remove the tokens length less than 2 again if some may appear after the above preprocessing
      text = ' '.join(text_ for text_ in text.split() if len(text_)>2)
      
      return  text


<>:13: DeprecationWarning: invalid escape sequence \.
<>:13: DeprecationWarning: invalid escape sequence \.
C:\Users\User\AppData\Local\Temp\ipykernel_14192\708893003.py:13: DeprecationWarning: invalid escape sequence \.
  text = re.sub('\.|,*','', text)


In [12]:
#there are 18494 nan plot summaries
movie_data.plot_summary.isna().sum()

18494

In [13]:
movie_topics = movie_data[pd.notnull(movie_data['plot_summary'])]

In [14]:
len(movie_topics.plot_summary) # after we drop the movies that doesn't have summaries

31291

In [15]:
movie_topics.plot_summary=movie_topics.plot_summary.apply(lambda x : clean_summary(x,stopwords=True, lemmatization=True))

C:\Users\User\AppData\Local\Temp\ipykernel_14192\2512033726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_topics.plot_summary=movie_topics.plot_summary.apply(lambda x : clean_summary(x,stopwords=True, lemmatization=True))


In [16]:
movie_topics.plot_summary=movie_topics.plot_summary.apply(lambda x : gensim.utils.simple_preprocess(str(x), deacc=True))


C:\Users\User\AppData\Local\Temp\ipykernel_14192\2042264280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_topics.plot_summary=movie_topics.plot_summary.apply(lambda x : gensim.utils.simple_preprocess(str(x), deacc=True))


In [17]:
summaries_corpus=movie_topics.plot_summary.values.tolist()

In [18]:
#
#count_vectors = CountVectorizer()
#x_counts = count_vectors.fit_transform(list(summaries_corpus))
#x_counts.todense()
#len(count_vectors.get_feature_names())
#tfidf_transformer = TfidfTransformer()
#x_tfidf = tfidf_transformer.fit_transform(x_counts)
#dimension = 5
#lda = LDA(n_components = dimension)
#lda_array = lda.fit_transform(x_tfidf)


In [19]:
summaries_corpus[:1]

[['horny',
  'college',
  'guy',
  'summer',
  'job',
  'cheerleader',
  'camp',
  'summer',
  'friendly',
  'naive',
  'michael',
  'mistaken',
  'homosexual',
  'isn',
  'world',
  'class',
  'jerk',
  'andy',
  'addiction',
  'masturbation',
  'try',
  'hook',
  'different',
  'girl',
  'night',
  'michael',
  'andy',
  'help',
  'head',
  'cheerleader',
  'sophie',
  'form',
  'cheerleading',
  'team',
  'group',
  'stripper',
  'end',
  'cheerleader',
  'michael',
  'andy',
  'run',
  'in',
  'local',
  'gentlemen',
  'club',
  'college',
  'scholarship',
  'offered',
  'group',
  'cheerleader',
  'win',
  'climatic',
  'team',
  'competition']]

In [20]:
#enrich our tokens by creating bigrams for each summary

def make_bigrams(summaries): 
    bigram = gensim.models.Phrases(summaries_corpus, min_count=5, threshold=100) # default parameters 
    bigram_model=gensim.models.phrases.Phraser(bigram) #create the model
    return [bigram_model[summary] for summary in summaries] # return the output of the bigram model for each summary

#for trigrams 
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
#trigram_model=gensim.models.phrases.Phraser(trigram) #create the model
#return [trigram_mod[bigram_model[summary]] for summary in summaries]


summaries_corpus=make_bigrams(summaries_corpus)

In [21]:
summaries_corpus[110]

['introductory',
 'storyline',
 'involves',
 'hitman',
 'lee',
 'wood',
 'dosmo',
 'pizzo',
 'kill',
 'roy',
 'foxx',
 'lie',
 'bed',
 'inject',
 'roy',
 'ex',
 'wife',
 'becky',
 'tranquilizer',
 'day',
 'afterward',
 'wood',
 'turn',
 'pizzo',
 'shooting',
 'blowing',
 'car',
 'order',
 'set',
 'fall',
 'guy',
 'wood',
 'flees',
 'scene',
 'norwegian',
 'girlfriend',
 'helga',
 'dosmo',
 'survives',
 'seeking',
 'shelter',
 'house',
 'allan',
 'hopper',
 'abrasive',
 'art_dealer',
 'suffers',
 'kidney',
 'stone',
 'take',
 'hopper',
 'abused',
 'assistant',
 'susan',
 'parish',
 'hostage',
 'completely',
 'unaware',
 'fact',
 'hopper',
 'called',
 'older',
 'sister',
 'audrey',
 'house',
 'medical',
 'help',
 'way',
 'brother',
 'house',
 'audrey',
 'nurse',
 'pick',
 'teddy',
 'pepper',
 'producer',
 'contemplating_suicide',
 'becky',
 'awakens',
 'horror',
 'discovers',
 'roy',
 'body',
 'bed',
 'run',
 'house',
 'flag',
 'vice',
 'detective',
 'happen',
 'driving',
 'initially',
 

In [22]:
#create the dictionary( a mapping between words and their integer ids) and corpus for LDA analaysis based on the preprocesseed summaries
dictionary = corpora.Dictionary(summaries_corpus)

#so for each word we have already mapped it in tuple of word id and frequency , so a summary is a bag of these tuples
corpus = [dictionary.doc2bow(summary) for summary in summaries_corpus]

In [23]:
len(summaries_corpus)

31291

In [24]:
len(dictionary)

120627

In [25]:
print(corpus[1])


[(6, 1), (18, 1), (31, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 5), (50, 1), (51, 2), (52, 2), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 3), (60, 1), (61, 1), (62, 5), (63, 3), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 3), (90, 1), (91, 1), (92, 1), (93, 1), (94, 10), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1), (111, 1), (112, 2), (113, 1), (114, 1), (115, 1), (116, 1), (117, 1), (118, 1), (119, 2), (120, 1), (121, 1), (122, 1), (123, 2), (124, 1), (125, 1), (126, 1), (127, 3), (128, 1), (129, 1), (130, 1), (131, 1), (132, 1), (133, 1), (134, 1), (135, 1), (136, 1), (137, 1), (138, 1), (139, 1), (140, 1), (141, 1), (142, 1), (143, 1), (144, 1), 

In [26]:
[[(dictionary[word_id],frequence) for word_id, frequence in cp] for cp in corpus[:1]]

[[('addiction', 1),
  ('andy', 3),
  ('camp', 1),
  ('cheerleader', 4),
  ('cheerleading', 1),
  ('class', 1),
  ('climatic', 1),
  ('college', 2),
  ('competition', 1),
  ('different', 1),
  ('end', 1),
  ('form', 1),
  ('friendly', 1),
  ('gentlemen_club', 1),
  ('girl', 1),
  ('group', 2),
  ('guy', 1),
  ('head', 1),
  ('help', 1),
  ('homosexual', 1),
  ('hook', 1),
  ('horny', 1),
  ('in', 1),
  ('isn', 1),
  ('jerk', 1),
  ('job', 1),
  ('local', 1),
  ('masturbation', 1),
  ('michael', 3),
  ('mistaken', 1),
  ('naive', 1),
  ('night', 1),
  ('offered', 1),
  ('run', 1),
  ('scholarship', 1),
  ('sophie', 1),
  ('stripper', 1),
  ('summer', 2),
  ('team', 2),
  ('try', 1),
  ('win', 1),
  ('world', 1)]]

In [27]:
#LDA implementation using the previous corpus 
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,# how often the model parameters should be updated
                                           chunksize=100, # number of summaries to be used in each training chunk
                                           passes=10, # training passes
                                           alpha='auto', # hyperparameter
                                           per_word_topics=True)
#so the LDA find the maximum likelihood distribution of words and summaries over topics(set to 10) 

In [28]:
lda_model.print_topics()

[(0,
  '0.007*"life" + 0.007*"father" + 0.007*"love" + 0.006*"film" + 0.006*"family" + 0.006*"mother" + 0.006*"friend" + 0.006*"find" + 0.005*"home" + 0.005*"tell"'),
 (1,
  '0.006*"men" + 0.005*"kill" + 0.004*"soldier" + 0.004*"order" + 0.004*"army" + 0.004*"killed" + 0.004*"war" + 0.004*"police" + 0.004*"village" + 0.004*"escape"'),
 (2,
  '0.007*"tom" + 0.007*"jack" + 0.006*"money" + 0.006*"car" + 0.006*"joe" + 0.006*"frank" + 0.005*"chris" + 0.005*"police" + 0.005*"town" + 0.005*"jim"'),
 (3,
  '0.010*"find" + 0.006*"kill" + 0.005*"escape" + 0.005*"tell" + 0.004*"body" + 0.004*"try" + 0.004*"man" + 0.004*"house" + 0.004*"car" + 0.004*"begin"')]

In [29]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=summaries_corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.304194553748708

Coherence Score:  0.3681100788222885


In [30]:
#interactive plot of topic representation 
#in the left each circle represent a topic and it's size determine the importance.Also in the right are the most representative words for each topic  
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
vis

C:\Users\User\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.103395  0.172215       1        1  44.898499
3     -0.126402  0.060031       2        1  25.328153
1     -0.097192 -0.238899       3        1  19.982444
2      0.326989  0.006653       4        1   9.790904, topic_info=        Term          Freq         Total Category  logprob  loglift
633     love  15496.000000  15496.000000  Default  30.0000  30.0000
171    money   8982.000000   8982.000000  Default  29.0000  29.0000
376   mother  13135.000000  13135.000000  Default  28.0000  28.0000
63       car   9961.000000   9961.000000  Default  27.0000  27.0000
151     kill  12471.000000  12471.000000  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
412     gang   2074.176772   4013.287925   Topic4  -5.5001   1.6637
38      team   1949.213006   4053.936334   Topic4  -5.5622   1.5915
1699    town   2301.752467   8098.365169   Topic4  -5.3960   1.0657
185   police   2451.132653  11240.577771   Topic4  -5.3331   0.8007
344      men   1598.713309   9904.598918   Topic4  -5.7604   0.4999

[276 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
5066       1  0.999596  affair
3870       2  0.041796   agent
3870       3  0.815015   agent
3870       4  0.143245   agent
5698       1  0.143675    alex
...      ...       ...     ...
1174       1  0.896359   young
1174       3  0.103601   young
6365       2  0.999674  zombie
12920      1  0.000853   zorro
12920      3  0.999008   zorro

[417 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 2, 3])

In [31]:
pyLDAvis.save_html(vis, 'lda.html')


In [32]:
#term to topics matrix
"""
topics_terms = lda_model.state.get_lambda() 
topics_terms_proba = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)
words = [lda_model.id2word[i] for i in range(topics_terms_proba.shape[1])]
pd.DataFrame(topics_terms_proba,columns=words)
"""

'\ntopics_terms = lda_model.state.get_lambda() \ntopics_terms_proba = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)\nwords = [lda_model.id2word[i] for i in range(topics_terms_proba.shape[1])]\npd.DataFrame(topics_terms_proba,columns=words)\n'

In [33]:
#doc/movies to topics distribution


In [34]:
len(summaries_corpus)

31291

In [35]:
train_vecs = []
for i in range(len(summaries_corpus)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(4)]
    train_vecs.append(topic_vec)

In [36]:
movie_topic_distribution = pd.DataFrame(train_vecs[0:], columns=['topic_1','topic_2','topic_3','topic_4'])


In [37]:
movie_topic_distribution

,topic_1,topic_2,topic_3,topic_4
0,0.633801,0.099370,0.236645,0.030183
1,0.165583,0.188212,0.245007,0.401198
2,0.801630,0.024238,0.107932,0.066200
3,0.635494,0.272188,0.073477,0.018841
4,0.378305,0.029088,0.236551,0.356057
...,...,...,...,...
31286,0.846602,0.098983,0.010952,0.043463
31287,0.632356,0.174139,0.080308,0.113198
31288,0.919264,0.058464,0.006877,0.015395
31289,0.729333,0.202205,0.009401,0.059061


In [41]:
topic_1 = movie_topic_distribution.topic_1.sort_values(ascending=False)
topic_2 = movie_topic_distribution.topic_2.sort_values(ascending=False)
topic_3 = movie_topic_distribution.topic_3.sort_values(ascending=False)
topic_4 = movie_topic_distribution.topic_4.sort_values(ascending=False)

In [48]:
topic_3

21715    0.690865
24403    0.646895
20014    0.644541
24169    0.616023
16267    0.613956
           ...   
7866     0.000538
5357     0.000487
18047    0.000432
29602    0.000344
25319    0.000319
Name: topic_3, Length: 31291, dtype: float32

In [43]:
topic1_most_representative_movie_ids=movie_topics.iloc[topic_1.index[:5]].movie_id
topic2_most_representative_movie_ids=movie_topics.iloc[topic_2.index[:5]].movie_id
topic3_most_representative_movie_ids=movie_topics.iloc[topic_3.index[:5]].movie_id
topic4_most_representative_movie_ids=movie_topics.iloc[topic_4.index[:5]].movie_id

In [44]:
topic1_most_representative_movie_ids.values

array([ 5045106,    77653, 12091274,  2014153,  7342730], dtype=int64)

In [45]:
topic2_most_representative_movie_ids.values

array([ 5826081, 33113634, 29677455, 28869936,   240483], dtype=int64)

In [46]:
topic3_most_representative_movie_ids.values

array([24121382, 24190384, 32852772, 25432586, 32173262], dtype=int64)

In [47]:
topic4_most_representative_movie_ids.values

array([10131383,  9930507, 10131333, 23617806, 14205601], dtype=int64)

In [58]:
movie_topic_distribution['movie_id']=movie_topics.movie_id.values

In [60]:
movie_topic_distribution

,topic_1,topic_2,topic_3,topic_4,movie_id
0,0.633801,0.099370,0.236645,0.030183,30332673
1,0.165583,0.188212,0.245007,0.401198,4213160
2,0.801630,0.024238,0.107932,0.066200,20624798
3,0.635494,0.272188,0.073477,0.018841,2250713
4,0.378305,0.029088,0.236551,0.356057,10331139
...,...,...,...,...,...
31286,0.846602,0.098983,0.010952,0.043463,1719500
31287,0.632356,0.174139,0.080308,0.113198,31799966
31288,0.919264,0.058464,0.006877,0.015395,13983035
31289,0.729333,0.202205,0.009401,0.059061,25818705


In [61]:
movie_topic_distribution.to_csv('movie_topic_distribution.csv', index=True)